In [1]:
import sys
sys.path.append('/home/server/gli-data-science/')

import ds_db
import pandas as pd
import numpy as np
import gensim
from multiprocessing import  Pool
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
q = '''

select tbtdr_ponta_user, tbtdr_tbto_id, tbtdr_rating, tbtdr_review 
from tb_transaction_delivery_rating ttdr 
where tbtdr_review notnull and tbtdr_review != ''
and tbtdr_created_date between '2021-06-01' and '2021-07-25'
'''

con, cur = ds_db.connect_prd_order_3()
df_re = pd.read_sql(q, con)


con.close()

You are connected


In [3]:
df_re = df_re[df_re['tbtdr_rating'].isin([1,2])]

In [4]:
from nlp_id.lemmatizer import Lemmatizer 
from nlp_id.stopword import StopWord 
from nlp_id.tokenizer import Tokenizer 

import re
import nltk

def basic_clean_single(text):
    lemmatizer = Lemmatizer() 
    
    stopword = StopWord() 
    stopwords = stopword.get_stopword() 
    
    tokenizer = Tokenizer() 
    words = tokenizer.tokenize(text)
    #return [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    return [lemmatizer.lemmatize(word) for word in words]

def basic_clean(df):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    df['tbtdr_review_token'] = df['tbtdr_review'].apply(basic_clean_single)
    return df


# processed_docs = list(df_re['tbtdr_review'].apply(basic_clean))

In [5]:
def parallelize_dataframe(df, func, n_cores=16):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

processed_docs = parallelize_dataframe(df_re, basic_clean)

In [7]:
processed_docs.head()

,tbtdr_ponta_user,tbtdr_tbto_id,tbtdr_rating,tbtdr_review,tbtdr_review_token
233,9990013350229355,6103590,2.0,Pembayaran Sulit,"[bayar, sulit]"
387,9990010138638665,5550827,1.0,"pesanan ini dibatalkan karna waktu kirim tdk sesuai dg estimasi, tetapi poin saya tetap terpotong","[pesan, ini, batal, karna, waktu, kirim, tdk, sesuai, dg, estimasi, , tetapi, poin, saya, tetap, potong]"
391,9990013149303785,1225562,2.0,"hanya whatsapp konfirmasi pesanan tanpa pemberitahuan produk ready semua atau ga dan hanya confirm shareloc, sesampainya pesanan malah yang dibutuhkan tidak ada. salut lah kecewanya","[hanya, whatsapp, konfirmasi, pesan, tanpa, beri, produk, ready, semua, atau, ga, dan, hanya, confirm, shareloc, , sampai, pesan, malah, yang, butuh, tidak, ada, , salut, lah, kecewa]"
495,9990013226669030,7101935,2.0,Produk rusak,"[produk, rusak]"
518,9990013357708115,7101676,1.0,"udh belanja, kok poin nya gk ada?","[udh, belanja, , kok, poin, nya, gk, ada, ]"


In [8]:

processed_docs = list(processed_docs['tbtdr_review_token'])
dictionary = gensim.corpora.Dictionary(processed_docs)

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)